In [ ]:
! jt -t grade3 -nf opensans

<h1><center>1. Load Data</center></h1>



In [ ]:
import os
import warnings
import pandas as pd

warnings.filterwarnings(action='ignore', message='numpy.dtype size changed')
warnings.filterwarnings(action='ignore', message='compiletime version 3.5 of module')

if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
os.chdir(os.path.split(workbookDir)[0])

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from synthesized.testing.evaluation import Evaluation
#evaluation = Evaluation(config_path='configs/evaluation/dataset_evaluation.json', name='james') # use this line if you want to run this notbook manually
evaluation = Evaluation()

In [ ]:
data = pd.read_csv(evaluation.config['data'])
data.dropna(inplace=True)
data.head(10)





<h1><center>2. Train model and generate synthetic data</center></h1>

In [ ]:
from sklearn.model_selection import train_test_split
from synthesized.core import BasicSynthesizer

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=0)

In [ ]:
def synthesize():
    with BasicSynthesizer(data=data, **evaluation.config['params']) as synthesizer:
        synthesizer.learn(data=train, num_iterations=evaluation.config['num_iterations'])
        return synthesizer.synthesize(n=len(test))

In [ ]:
synthesized = [synthesize() for i in range(evaluation.config['num_passes'])]

## Display aggregated statistics 

In [ ]:
from scipy.stats import ks_2samp
import numpy as np
import seaborn as sns

In [ ]:
synthesizer = BasicSynthesizer(data=data)

In [ ]:
def plot_avg_distances():
    test_ = synthesizer.preprocess(test.copy())
    result = []
    for i, s in enumerate(synthesized):
        synthesized_ = synthesizer.preprocess(s.copy())
        distances = [ks_2samp(test_[col], synthesized_[col])[0] for col in synthesized_.columns]
        avg_distance = np.mean(distances)
        print('run: {}, AVG distance: {}'.format(i+1, avg_distance))
        result.append({'run': i+1, 'avg_distance': avg_distance})
        evaluation['avg_distance'] = avg_distance
    df = pd.DataFrame.from_records(result)
    df['run'] = df['run'].astype('category')
    g = sns.barplot(y='run', x='avg_distance', data=df)
    g.set_xlim(0.0, 1.0)

In [ ]:
plot_avg_distances()

## Details for the 1st Run 

In [ ]:
from synthesized.testing import UtilityTesting
testing = UtilityTesting(synthesizer, train, test, synthesized[0])

In [ ]:
testing.show_distribution_distances()

In [ ]:
testing.show_distributions()

## Display correlations

In [ ]:
testing.show_correlation_diffs(threshold=0.2)

In [ ]:
testing.show_corr_matrices()

## Demonstrate the utility for training ML models

In [ ]:
evaluation['utility'] = testing.utility(target=evaluation.config['target'])

In [ ]:
evaluation.write_metrics()